In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
sys.path.append("../src")
import os
from src.DS_Generator import DataSet_Generator
from src.model_evaluation import mean_average_precision
from src.data_augmentation import *
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
from util import *
import datetime
%load_ext tensorboard

Okay Louis, such dir eine Batchsize aus. Bei mir hat nur 32 keine fehler verusacht.

In [2]:
# Für deinen Model order
if "models" not in os.listdir("../../"):
    os.makedirs("../../models")
    
BATCH_SIZE = 32

In [3]:
train_ds, val_ds = DataSet_Generator().generate_species_data(TRAIN_SPECIES_DF, augment=1, batch_size=BATCH_SIZE)
num_classes = len(set(TRAIN_SPECIES_DF["species"]))

We threw away the datapoint with index 31680 


2022-03-28 18:09:56.282677: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
Input = tf.keras.Input((224,224,3))
base = tf.keras.applications.inception_v3.InceptionV3(weights="imagenet", include_top=False,input_tensor=Input)
flatten = base.output
flatten = tf.keras.layers.Flatten()(flatten)
head = tf.keras.layers.Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.l2(0.001))(flatten)
head = tf.keras.layers.Dense(128, activation="relu",kernel_regularizer=tf.keras.regularizers.l2(0.001))(head)
head = tf.keras.layers.Dense(64, activation="relu",kernel_regularizer=tf.keras.regularizers.l2(0.001))(head)
head = tf.keras.layers.Dense(num_classes,activation="softmax")(head)
model = tf.keras.Model(inputs=Input, outputs=head,name="Inception_v3_with_imagenet_weights_and_l2")

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),loss="categorical_crossentropy",metrics=["acc"])

if model.name not in os.listdir("../../models/"):
    os.makedirs("../../models/"+model.name)
    os.makedirs("../../models/"+model.name+"/logs")
    os.makedirs("../../models/"+model.name+"/saves")
    
time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

log_dir = "../../models/"+model.name+"/logs/" +time_stamp 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = "../../models/"+model.name+"/saves/" +time_stamp +"/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

In [6]:
model.fit(
train_ds,
epochs=15,
validation_data=val_ds,

allbacks=[cp_callback,tensorboard_callback])

Epoch 1/15
1436/1436 [==============================] - ETA: 0s - loss: 1.5609 - acc: 0.6327
Epoch 1: saving model to ../../models/Inception_v3_with_imagenet_weights_and_l2/saves/20220328-180959/cp-0001.ckpt
1436/1436 [==============================] - 6716s 5s/step - loss: 1.5609 - acc: 0.6327 - val_loss: 1.3650 - val_acc: 0.6708
Epoch 2/15
1436/1436 [==============================] - ETA: 0s - loss: 0.9123 - acc: 0.7710
Epoch 2: saving model to ../../models/Inception_v3_with_imagenet_weights_and_l2/saves/20220328-180959/cp-0002.ckpt
1436/1436 [==============================] - 6567s 5s/step - loss: 0.9123 - acc: 0.7710 - val_loss: 0.9819 - val_acc: 0.7515
Epoch 3/15
1436/1436 [==============================] - ETA: 0s - loss: 0.7174 - acc: 0.8231
Epoch 3: saving model to ../../models/Inception_v3_with_imagenet_weights_and_l2/saves/20220328-180959/cp-0003.ckpt
1436/1436 [==============================] - 6390s 4s/step - loss: 0.7174 - acc: 0.8231 - val_loss: 0.7778 - val_acc: 0.8015
E

KeyboardInterrupt: 